In [ ]:
!pip install pyngrok
from pyngrok import ngrok

# Set the auth token
ngrok.set_auth_token("Your_Ngrok_Token")


In [ ]:
import torch
torch.cuda.empty_cache()


In [ ]:
# --- Install dependencies ---
!pip uninstall -y whisper
!pip install git+https://github.com/openai/whisper.git
!pip install torch flask flask-cors pyngrok gTTS > /dev/null


In [ ]:
# --- Imports ---
import whisper

# --- Load Whisper STT model ---
print("⏳ Loading Whisper model...")
whisper_model = whisper.load_model("small.en")
print("✅ Whisper 'small.en' model is loaded successfully.\n")


⏳ Loading Whisper model...


100%|███████████████████████████████████████| 461M/461M [00:22<00:00, 21.3MiB/s]


✅ Whisper 'small.en' model is loaded successfully.



In [ ]:
import os
import traceback
import whisper
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
from pyngrok import ngrok
from gtts import gTTS

# --- Flask setup ---
app = Flask(__name__)
CORS(app)

# --- Create temp folders ---
os.makedirs("temp_audio", exist_ok=True)
os.makedirs("temp_text", exist_ok=True)

# --- Speech-to-Text Route ---
@app.route("/transcribe", methods=["POST"])
def transcribe():
    print("\n📩 Received /transcribe request")
    try:
        if "file" not in request.files:
            return jsonify({"error": "No file uploaded"}), 400

        audio_file = request.files["file"]
        path = "temp_audio/input.webm"
        audio_file.save(path)
        print(f"✅ Audio saved to {path}")

        result = whisper_model.transcribe(path)
        text = result["text"].strip()
        print(f"📝 Transcription: {text}")

        with open("temp_text/output.txt", "w") as f:
            f.write(text)

        return jsonify({"text": text})

    except Exception as e:
        traceback.print_exc()
        return jsonify({"error": str(e)}), 500


# --- Text-to-Speech Route ---
@app.route("/speak", methods=["POST"])
def speak():
    print("\n📩 Received /speak request")
    try:
        text = request.form.get("text", "")
        if not text:
            return jsonify({"error": "No text provided"}), 400

        print(f"🗣 Converting text to speech: {text}")
        tts = gTTS(text)
        audio_path = "temp_audio/output.mp3"
        tts.save(audio_path)
        print(f"💾 TTS saved: {audio_path}")

        return send_file(audio_path, mimetype="audio/mpeg")

    except Exception as e:
        traceback.print_exc()
        return jsonify({"error": str(e)}), 500


# --- Start Ngrok ---
public_url = ngrok.connect(5000)
print(f"\n🌐 NGROK URL: {public_url}\n")

# --- Run Flask app ---
app.run(port=5000)
